In [1]:
!pip install langchain langchain-huggingface

In [2]:

!pip install google-generativeai


In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
!pip install -U langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [5]:
!pip install langchain langchain-huggingface sentence-transformers

In [6]:
!pip install beautifulsoup4

In [23]:
!pip install langchain-groq

In [7]:
#Creating a function that will easily help us in converting a webpage to a document file



import requests

def save_webpage_to_txt(url, filename="Artificial_Intelegence.txt"):
    """
    Fetches the content of a web page and saves it to a text file.

    Args:
        url (str): The URL of the web page to fetch.
        filename (str, optional): The name of the output text file.
                                  Defaults to "webpage_content.txt".
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes



        #For extracting only visible text (requires a parser like BeautifulSoup)
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.get_text()

        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Web page content from {url} saved to {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the web page: {e}")
    except IOError as e:
        print(f"Error writing to file: {e}")
    except ImportError:
        print("Please install BeautifulSoup: pip install beautifulsoup4")

In [8]:
# Loading a link from the web to use as document to be used in our RAG pipeline
webpage_url = "https://cloud.google.com/learn/what-is-artificial-intelligence"
output_filename = "Artificial_Intelegence.txt"
save_webpage_to_txt(webpage_url, output_filename)

Web page content from https://cloud.google.com/learn/what-is-artificial-intelligence saved to Artificial_Intelegence.txt


In [24]:
#activating key
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API')

#DATA INGESTION


In [9]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader('/content/Artificial_Intelegence.txt', encoding="utf8")
data = loader.load()


In [10]:
print(data[0].page_content)

What Is Artificial Intelligence (AI)? | Google CloudPage ContentsTopicsWhat is Artificial Intelligence?What is Artificial Intelligence (AI)?Artificial intelligence (AI) is a set of technologies that enable computers to perform a variety of advanced functions, including the ability to see, understand and translate spoken and written language, analyze data, make recommendations, and more. AI is the backbone of innovation in modern computing, unlocking value for individuals and businesses. For example, optical character recognition (OCR) uses AI to extract text and data from images and documents, turns unstructured content into business-ready structured data, and unlocks valuable insights.  Ready to get started? New customers get $300 in free credits to spend on Google Cloud.Get started for freeStay informed22:01Introduction to generative AI Artificial intelligence definedArtificial intelligence is a field of science concerned with building computers and machines that can reason, learn, a

#Chunking of the Data

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks=text_splitter.split_documents(data)





In [12]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Specify the desired model from Hugging Face Hub
model_name = "sentence-transformers/all-mpnet-base-v2"

# Initialize the embeddings object
embeddings = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from huggingface_hub import login
from google.colab import userdata
api_key = userdata.get('HUGGING_FACE_API')
login(token=api_key)

In [26]:
import os
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model="qwen/qwen3-32b"
)



In [14]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
vector_store = InMemoryVectorStore(embedding=embeddings)
index = vector_store.add_documents(documents=text_chunks)

In [15]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})


In [ ]:
#print(retrieved_docs)

In [36]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
dont say stuff like <think>\nOkay, the user is asking "what is AI". Let me look at the provided context.
\n\nFirst documen
dont say stuff like according to the context,base on the content
just give the answer
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [37]:
prompt=ChatPromptTemplate.from_template(template)

In [18]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [19]:

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [20]:
output_parser=StrOutputParser()

In [38]:

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser



rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [39]:
rag_chain.invoke("what is ai")

'<think>\nOkay, the user is asking "what is AI". Let me look at the provided context.\n\nThe first document says AI is a field of science focused on building computers and machines that can reason, learn, and act in ways requiring human intelligence. It mentions that AI deals with data scales beyond human analysis and involves disciplines like computer science, data analytics, engineering, linguistics, etc.\n\nThe second document from Google Cloud defines AI as technologies enabling computers to perform advanced functions like understanding language, analyzing data, making recommendations. It\'s the backbone of modern computing innovations for individuals and businesses.\n\nThe third document adds that AI uses machine learning and deep learning for business applications like data analytics, predictions, object categorization, NLP, and intelligent data retrieval. It works by learning from large data sets to identify patterns.\n\nPutting this all together, AI is a broad field aiming to c